In [13]:
# Import libraries
# Standard libraries for reading pdf
import PyPDF2 as pypdf
import PyPDF2
import re
import requests
import io
from io import BytesIO
import concurrent.futures
from itertools import groupby
import random
import string
import pandas as pd
import time
import praw

# Use open parse for reading the tables
import openparse
from bs4 import BeautifulSoup

# Llama libraries for RAG
from llama_index.core import (
    Document,
    Prompt,
    VectorStoreIndex,
    
)
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP

# For the responses
from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
    CBEventType,
)
from llama_index.core import ServiceContext
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

from llama_index.core.retrievers import VectorIndexRetriever

In [15]:
# Standard authentication
reddit_read_only  = praw.Reddit(
    client_id="client_id",
    client_secret="client_secret",
    user_agent="user_agent",
)

In [16]:
subreddit = reddit_read_only.subreddit("OMSCS")
 # Display the name of the Subreddit
print("Display Name:", subreddit.display_name)
 # Display the title of the Subreddit
print("Title:", subreddit.title)

Display Name: OMSCS
Title: Georgia Tech OMS Computer Science


In [18]:
#posts  = subreddit.flair('Registration')
posts = subreddit.top("month")
# Scraping the top posts of the current month
 
posts_dict = {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Post URL": [],
              "Flair": []
              }
 
for post in posts:
    # Title of each post
    posts_dict["Title"].append(post.title)
     
    # Text inside a post
    posts_dict["Post Text"].append(post.selftext)
     
    # Unique ID of each post
    posts_dict["ID"].append(post.id)
     
    # The score of a post
    posts_dict["Score"].append(post.score)
     
    # Total number of comments inside the post
    posts_dict["Total Comments"].append(post.num_comments)
     
    # URL of each post
    posts_dict["Post URL"].append(post.url)
    
    # Get the flair of the post
    posts_dict["Flair"].append(post.link_flair_text)
    
top_posts = pd.DataFrame(posts_dict)
top_posts

C:\Users\UGGTech\AppData\Local\Temp\ipykernel_80204\1678316060.py:2: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  posts = subreddit.top("month")


,Title,Post Text,ID,Score,Total Comments,Post URL,Flair
0,CS6601 AI: Incomplete grades handed out based ...,The CS6601 Spring 2024 finals had a lot of iss...,1cqqmng,190,73,https://www.reddit.com/r/OMSCS/comments/1cqqmn...,CS 6601 AI
1,A tale of two students - this could apply to s...,,1d3lcw2,147,59,https://i.redd.it/797pbb1t8f3d1.png,This is a Meme
2,This has been a humbling experience,I enrolled in this program in Fall 2023. Dropp...,1d6phxr,111,60,https://www.reddit.com/r/OMSCS/comments/1d6phx...,Withdrawal
3,My Georgia Tech OMSCS Review - Reflections fro...,,1d40zmr,111,44,https://www.ddanieltan.com/posts/omscs-review/,I GOT OUT :joyner-shocked:
4,OMSCS T-Shirt arrived. Have you guys received ...,I entered omscs on fall 2022. It looks good. I...,1d9qcwg,98,33,https://i.redd.it/ypsdclko305d1.jpeg,Social
...,...,...,...,...,...,...,...
95,switching to OMSCS after core classes at OSU?,I am in a bit of a predicament deciding betwee...,1d5vzdn,8,8,https://www.reddit.com/r/OMSCS/comments/1d5vzd...,Deferment
96,Has anyone taken the CS 8001 ODA seminar to pr...,EDIT: i mean cs8001 OLP seminar NOT CS8001 ODA...,1d4i0lt,7,2,https://www.reddit.com/r/OMSCS/comments/1d4i0l...,Courses
97,CS/Programming foundations before OMSCS,"I’m gearing up for the OMSCS program, but I’m ...",1cy79mr,6,6,https://www.reddit.com/r/OMSCS/comments/1cy79m...,Admissions
98,What are the best courses to list if you have ...,I have graduated with non-stem bachelor degree...,1cur7gg,7,9,https://www.reddit.com/r/OMSCS/comments/1cur7g...,Courses


In [19]:
submission = reddit_read_only.submission("1cqqmng")

comments_dict = {"Post Text": [],
              "ID": [], "Score": [],
              }
 
for comments in submission.comments:     
    # Text inside a post
    comments_dict["Post Text"].append(comments.body)
     
    # Unique ID of each post
    comments_dict["ID"].append(comments.id)
     
    # The score of a post
    comments_dict["Score"].append(comments.score)
     
    
comments_df = pd.DataFrame(comments_dict)
comments_df

,Post Text,ID,Score
0,6601 was pretty fanatically focused on cheatin...,l3t57oe,89
1,I cannot upvote this enough. I was enrolled in...,l3u6ci6,36
2,Absolutely true. What happened this semester w...,l3ucss8,26
3,Will definitely not be taking this course. Tha...,l3udy36,27
4,You should report this to the faculty with all...,l3ufpil,19
5,Years ago I read the syllabus and chose the GA...,l3ufyw8,15
6,Didn't KBAI give everyone 0's on peer reviews ...,l3vfw1u,12
7,I totally feel you. I took this course in 2020...,l3ti579,25
8,So the students are the beta testers?,l3u921y,11
9,Good to hear that AI is STILL an absolute mess...,l3uwqvp,10


In [20]:
# Not sure if we want to deal with the sentiment of the text as well
from textblob import TextBlob

def compute_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

comments_df['sentiment'] = comments_df.apply(lambda row: compute_sentiment(row['Post Text']), axis=1)

In [21]:
data_dict = comments_df.to_dict(orient='records')

[{'Post Text': "6601 was pretty fanatically focused on cheating.\n\nCheating is bad and you shouldn't do it, sure, but definitely felt strangely intense in that class.",
  'ID': 'l3t57oe',
  'Score': 89,
  'sentiment': 0.05000000000000003},
 {'Post Text': 'I cannot upvote this enough. I was enrolled in this class this past semester, and ended up withdrawing from it during the midterm, purely due to the mismanagement and confusion associated with it. \n\nAdditionally, the hyper focus on cheating left me feeling paranoid all the time. Definitely felt like they were constantly out to get me. Since this is my first class, I was afraid the whole program would be like that.',
  'ID': 'l3u6ci6',
  'Score': 36,
  'sentiment': -0.05833333333333332},
 {'Post Text': 'Absolutely true. What happened this semester was so far past acceptable, I sincerely hope that a further investigation goes into how this class operates, and changes will be made to how it\'s run. Many reviews and feedback from peopl

In [22]:
text_embedding_model = "thenlper/gte-base"
# Instantiate LLM and embedding model
embed_model = HuggingFaceEmbedding(model_name=text_embedding_model)
#model = 'llama-2-7b-chat.Q4_K_M.gguf'
model = 'ggml-model-Q4_K_M.gguf'

llm = LlamaCPP(model_path=model, temperature=0, max_new_tokens=256, context_window=4096, verbose=True)

C:\Users\UGGTech\anaconda3\envs\RAG_Env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ggml-model-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = model
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32

In [23]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
text_splitter = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=256
)
service_context = ServiceContext.from_defaults(
    callback_manager=callback_manager,
    llm=llm,
    embed_model=embed_model,
    text_splitter=text_splitter,
)

C:\Users\UGGTech\AppData\Local\Temp\ipykernel_80204\3092009722.py:7: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [25]:
text_nodes = []
for entry in data_dict:
    text_node = TextNode(
        text=entry['Post Text'],
        id_=entry['ID'],
        metadata={
            'score': entry['Score'],
            'sentiment': entry['sentiment']
        }
    )
    text_nodes.append(text_node)

print("Text nodes with metadata added to the vector store successfully!")

index = VectorStoreIndex(
    text_nodes,
    service_context=service_context,
    embed_model=embed_model,
    llm=llm
)

Text nodes with metadata added to the vector store successfully!
**********
Trace: index_construction
**********


In [26]:
%%time
# Query engine with streaming response and top 1 document filter for efficiency
response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.COMPACT,
    service_context=service_context,
    streaming=True
)
query_engine = index.as_query_engine(
    response_synthesizer=response_synthesizer,
    similarity_top_k=1,
    streaming=True
)

CPU times: total: 78.1 ms
Wall time: 313 ms


In [28]:
%%time
response = query_engine.query("Summerize the class CS6001 and if it worth taking")
response.print_response_stream()

Llama.generate: prefix-match hit


 The class CS6001 was used as guinea pigs for a cheating detection system in summer of 2022 which is negligent and should have repercussions. It would be better to take the class in a sandbox to compare with existing systems and require consent. 
It's not clear whether the class is worth taking or not based on this information.


llama_print_timings:        load time =   22047.00 ms
llama_print_timings:      sample time =      55.71 ms /    69 runs   (    0.81 ms per token,  1238.60 tokens per second)
llama_print_timings: prompt eval time =    1025.81 ms /     9 tokens (  113.98 ms per token,     8.77 tokens per second)
llama_print_timings:        eval time =   18355.46 ms /    68 runs   (  269.93 ms per token,     3.70 tokens per second)
llama_print_timings:       total time =   20126.14 ms /    77 tokens


CPU times: total: 25.8 s
Wall time: 21.2 s
